In [4]:
!pip install -q transformers[sentencepiece] fastbook fastai ohmeow-blurr nbdev

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [6]:
df = pd.read_csv("/kaggle/input/moviescombinedgenerid-csv/moviescombinedGenerid.csv")
df.head()

,youtubeId,movieId,title,genres,youtube_link,genre_ids,transcription
0,K26_sDKnvMU,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://www.youtube.com/watch?v=K26_sDKnvMU,"[1, 2, 3, 8]",sergeant yes sir establish your recon post downstairs Code Red repeat we are at Code Red recon plan charlie execute move move move it's a Walt Disney Pictures presents a totally new animated motion picture event Star Command come in do you read me the story of two toys there seems to be no sign of intelligent life anywhere hello oh yeah headed for a showdown my name is woody this is my spot I am Buzz Lightyear I come in peace you are a child's plaything you are a sad strange little man and playing by their own rules draw me again I don't like confrontations Vulcan alien where your monkey m...
1,3LPANjHlPxo,2,Jumanji (1995),Adventure|Children|Fantasy,https://www.youtube.com/watch?v=3LPANjHlPxo,"[1, 8]",I'm glad you decided to buy this place I'm sure that you and your kids will be very happy here every house has Secrets everybody in town thinks the place is haunted but what happened in this house little Allan Parish he just vanished about uh 25 years ago is a mystery that began a long time ago When Alan Parish was just a young boy who made an incredible Discovery Jumanji you want to play Al look in the jungle you must wait until the dice read five or eight what's happening to [Music] you 26 years later Peter and Judy Shephard are about to play the same game and make an incredible discover...
2,rEnOoWs3FuA,3,Grumpier Old Men (1995),Comedy|Romance,https://www.youtube.com/watch?v=rEnOoWs3FuA,"[3, 13]",NaN
3,j9xml1CxgXI,4,Waiting to Exhale (1995),Comedy|Drama|Romance,https://www.youtube.com/watch?v=j9xml1CxgXI,"[3, 6, 13]",NaN
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995),Comedy,https://www.youtube.com/watch?v=ltwvKLnj1B4,[3],NaN


In [7]:
df.shape

(25623, 7)

In [8]:
df = df.dropna(subset=['transcription'])
columns_to_drop = ['youtubeId', 'movieId', 'title', 'genre_ids', 'youtube_link']
df = df.drop(columns=columns_to_drop)

In [9]:
df['genres'] = df['genres'].apply(lambda x: x.split('|'))
df.head()

,genres,transcription
0,"[Adventure, Animation, Children, Comedy, Fantasy]",sergeant yes sir establish your recon post downstairs Code Red repeat we are at Code Red recon plan charlie execute move move move it's a Walt Disney Pictures presents a totally new animated motion picture event Star Command come in do you read me the story of two toys there seems to be no sign of intelligent life anywhere hello oh yeah headed for a showdown my name is woody this is my spot I am Buzz Lightyear I come in peace you are a child's plaything you are a sad strange little man and playing by their own rules draw me again I don't like confrontations Vulcan alien where your monkey m...
1,"[Adventure, Children, Fantasy]",I'm glad you decided to buy this place I'm sure that you and your kids will be very happy here every house has Secrets everybody in town thinks the place is haunted but what happened in this house little Allan Parish he just vanished about uh 25 years ago is a mystery that began a long time ago When Alan Parish was just a young boy who made an incredible Discovery Jumanji you want to play Al look in the jungle you must wait until the dice read five or eight what's happening to [Music] you 26 years later Peter and Judy Shephard are about to play the same game and make an incredible discover...
5,"[Action, Crime, Thriller]",[Music] he's here I can feel it you search for the scent of your prey and then you hunt them down that's the only thing you're committed to it keeps me sharp on the edge where I got to be you want to be making moves on the street allow nothing to be in your life that you cannot walk out on in 30 seconds flat if you spot the heat around the corner no my life's a disaster so because I spend all my time chasing guys like you around the block I do what I do best I take scores you do what you do best trying to stop guys like me youve been walking through our life Dead all I am is what I'm going...
7,"[Adventure, Children]",and where do you think you're going I'm just running away from home again Jonathan Taylor [Music] Thomas and Brad Renfro are two guys who go looking for adventure but when your names are Tom Sawyer hey Becky hey and hu Finn engine Joe's on the move I ain't scared of nothing a No Ordinary Adventure is going to do it's a treasure map what if we could get the [Music] map wish I could see him just one more time they think I'm dead I kiss him right in front of everybody the original Bad Boys Are Back hey Becky Tom and HW
9,"[Action, Adventure, Thriller]",when the world is the target 72 hours ago a secret weapon system was detonated over 7:00 a.m. and the threat is real don't deny exists a radiation surge that destroys everything with an electronic circuit you can still depend on one man I wouldn't to find Goldeneye three find who took it to will stop it one names bond James Bond the world's most famous secret agent is back playing - please and this time double-oh-seven is facing the ultimate enemy the man who knows him best hello James what an unpleasant surprise double-o 6 what's the message no pithy comeback it was a friend I know is you...


In [10]:
genres_list = df.genres.to_list()
genre_count = {}
for genres in genres_list:
  print(genres)
  for genre in genres:
    if genre in genre_count.keys():
      genre_count[genre] += 1
    else:
      genre_count[genre] = 1
print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
['Adventure', 'Children', 'Fantasy']
['Action', 'Crime', 'Thriller']
['Adventure', 'Children']
['Action', 'Adventure', 'Thriller']
['Comedy', 'Drama', 'Romance']
['Drama']
['Crime', 'Drama']
['Comedy', 'Crime', 'Thriller']
['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller']
['Drama', 'Sci-Fi']
['Drama', 'Romance']
['Children', 'Drama']
['Drama', 'Romance']
['Drama']
['Mystery', 'Sci-Fi', 'Thriller']
['Children', 'Drama']
['Comedy', 'Romance']
['Drama', 'War']
['Drama']
['Action', 'Adventure', 'Fantasy']
['Mystery', 'Thriller']
['Animation', 'Children', 'Drama', 'Musical', 'Romance']
['Crime', 'Mystery', 'Thriller']
['Children', 'Comedy']
['Drama', 'Thriller']
['Drama']
['Comedy', 'Crime']
['Action']
['Action', 'Sci-Fi', 'Thriller']
['Action', 'Crime', 'Drama', 'Thriller']
['Drama', 'Thriller']
['Crime', 'Drama', 'Romance']
['Comedy', 'Drama']
['Comedy', 'Drama']
['Drama', 'Romance']
['Children', 'Comedy']
['Comedy']
['Drama', 

In [11]:
threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres), rare_genres[:5]

(1, ['IMAX'])

In [12]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(genre_count.items())}
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

In [13]:
from ast import literal_eval


In [14]:
# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
genres_list = df.genres.to_list()

for all_genres in genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in all_genres:
    genre_type_index = encode_genre_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

categorical_genre_list[3][:23]

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
df['genre_cat_list'] = categorical_genre_list
df.head()

,genres,transcription,genre_cat_list
0,"[Adventure, Animation, Children, Comedy, Fantasy]",sergeant yes sir establish your recon post downstairs Code Red repeat we are at Code Red recon plan charlie execute move move move it's a Walt Disney Pictures presents a totally new animated motion picture event Star Command come in do you read me the story of two toys there seems to be no sign of intelligent life anywhere hello oh yeah headed for a showdown my name is woody this is my spot I am Buzz Lightyear I come in peace you are a child's plaything you are a sad strange little man and playing by their own rules draw me again I don't like confrontations Vulcan alien where your monkey m...,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[Adventure, Children, Fantasy]",I'm glad you decided to buy this place I'm sure that you and your kids will be very happy here every house has Secrets everybody in town thinks the place is haunted but what happened in this house little Allan Parish he just vanished about uh 25 years ago is a mystery that began a long time ago When Alan Parish was just a young boy who made an incredible Discovery Jumanji you want to play Al look in the jungle you must wait until the dice read five or eight what's happening to [Music] you 26 years later Peter and Judy Shephard are about to play the same game and make an incredible discover...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,"[Action, Crime, Thriller]",[Music] he's here I can feel it you search for the scent of your prey and then you hunt them down that's the only thing you're committed to it keeps me sharp on the edge where I got to be you want to be making moves on the street allow nothing to be in your life that you cannot walk out on in 30 seconds flat if you spot the heat around the corner no my life's a disaster so because I spend all my time chasing guys like you around the block I do what I do best I take scores you do what you do best trying to stop guys like me youve been walking through our life Dead all I am is what I'm going...,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"[Adventure, Children]",and where do you think you're going I'm just running away from home again Jonathan Taylor [Music] Thomas and Brad Renfro are two guys who go looking for adventure but when your names are Tom Sawyer hey Becky hey and hu Finn engine Joe's on the move I ain't scared of nothing a No Ordinary Adventure is going to do it's a treasure map what if we could get the [Music] map wish I could see him just one more time they think I'm dead I kiss him right in front of everybody the original Bad Boys Are Back hey Becky Tom and HW,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,"[Action, Adventure, Thriller]",when the world is the target 72 hours ago a secret weapon system was detonated over 7:00 a.m. and the threat is real don't deny exists a radiation surge that destroys everything with an electronic circuit you can still depend on one man I wouldn't to find Goldeneye three find who took it to will stop it one names bond James Bond the world's most famous secret agent is back playing - please and this time double-oh-seven is facing the ultimate enemy the man who knows him best hello James what an unpleasant surprise double-o 6 what's the message no pithy comeback it was a friend I know is you...,"[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [16]:
df.to_csv("Movies_categorical_data.csv", index=False)

# Dataloaders and Modeling

In [17]:
labels = list(encode_genre_types.keys())

In [18]:
model_name = "distilroberta-base"
model_cls = AutoModelForSequenceClassification
config = AutoConfig.from_pretrained(model_name)
config.num_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(model_name, model_cls=model_cls, config=config)
hf_model.config.problem_type = "multi_label_classification"

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

In [19]:
blocks = (TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), MultiCategoryBlock(encoded=True,vocab=labels))
dblock = DataBlock(blocks=blocks, get_x=ColReader('transcription'), get_y=ColReader('genre_cat_list'), splitter=RandomSplitter(valid_pct=0.1, seed=42))

In [20]:
dls = dblock.dataloaders(df, bs=16)
torch.save(dls, "dls-multilabel-movie-classifier.pkl")


In [21]:
# prompt: save this model to drive

# from google.colab import drive
# drive.mount('/content/drive')
!cp dls-multilabel-movie-classifier.pkl /kaggle/working/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


cp: 'dls-multilabel-movie-classifier.pkl' and '/kaggle/working/dls-multilabel-movie-classifier.pkl' are the same file


In [22]:
dls.vocab

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Action', 'Crime', 'Thriller', 'Drama', 'Romance', 'Horror', 'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'Film-Noir', 'Western', 'IMAX']

In [23]:
dls.show_batch(dataloaders=dls, max_n=4, trunc_at=512)

,text,target
0,"[strings playing\n""all along the watchtower""] Male announcer: ladies\nand gentlemen, welcome. Today it is\nmy distinct pleasure To introduce a man\nwho is recognized-- The battle over evolution\nis only one skirmish In a much larger war. Science simply makes no use\nof the hypothesis of God. Ask yourself, what has\nintelligent design given us?\nNothing. We cannot accept\nintelligent design As an alternative\nscientific theory. They will never accept\nthat we have a better argument. They just pester us,\nand they waste",[Documentary]
1,[Music] [Music] [Laughter] [Music] oh [Music] [Music] oh [Music] way [Music] [Music] Manhattan Beach Police Department crime report 00 6799 Dr number 8501 713 reporting officer detective J hog date of report 8283 person filing complaint judianne Johnson misses report follows Mrs Johnson came to the police station approximately 10:30 a.m. with her son Malcolm aged 2 and 1/2 years and filed a complaint charging child sexual abuse reporting officer conducted an interview with the child and with his mother Mrs,"[Thriller, Drama]"
2,[Applause] so [Music] river city junction river city next station stop river city next [Music] it's some big new department stores back east they let folks buy anything on credit if i don't give credit i'm gonna lose customers you're crazy with the heat credit is no good for an ocean salesman why not what's the matter with credit it's old-fashioned charlie you're an anvil salesman your firm give credit no sir nor anybody else cash for the merchandise cash for the button hooks cash for the cotton goods cash,"[Comedy, Drama, Romance, Musical]"
3,you you [Music] [Music] [Music] I jumped out of the West but oh my god the planes gonna crash this thing came right across north to south of incidentally I was a private pilot so I have some familiarity with there it glided over my house and there was I saw what I thought to be a kind of a boomerang shape oh my god that's a wingspan I've never seen anything like that they were reported by at least one commercial airline pilot it was you fungus product it passed right in front of us just about right above e,[Documentary]


In [25]:
model = BaseModelWrapper(hf_model)

acc_02 = partial(accuracy_multi, thresh=0.2)

learner = Learner(dls,
                  model,
                  opt_func=partial(OptimWrapper, opt=torch.optim.AdamW),
                  loss_func=BCEWithLogitsLossFlat(),
                  metrics=[acc_02],
                  cbs=[BaseModelCallback],
                  splitter=blurr_splitter
                  ).to_fp16()

In [26]:
learner.freeze()

In [ ]:
learner.fit_one_cycle(2,4.75e-4)

/opt/conda/lib/python3.10/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/conda/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy_multi,time
0,0.252274,0.235198,0.873086,1:18:52


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling pa

learner.lr_find(suggest_funcs=[slide, valley])

In [26]:
learner.save("movie-classifier-stage-0")

Path('models/movie-classifier-stage-0.pth')

In [27]:
learner.export("models/movie-classifier-stage-0.pkl")

In [30]:
learner.unfreeze()

In [33]:
learner.fit_one_cycle(3,slice(4.75e-3, 2.25e-5))

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.246404,0.261733,0.861364,00:43
1,0.209655,0.264121,0.861962,00:42
2,0.146190,0.247417,0.877512,00:45


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

  self.pid = os.fork()


In [34]:
learner.save("movie-classifier-stage-1")

Path('models/movie-classifier-stage-1.pth')

In [35]:
learner.export("models/movie-classifier-stage-1.pkl")

In [36]:
learner.unfreeze()

In [37]:
learner.fit_one_cycle(7,slice(4.75e-3, 2.25e-5))

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.132498,0.261900,0.874163,00:43
1,0.176197,0.308659,0.850000,00:44
2,0.164807,0.280247,0.867823,00:43
3,0.121302,0.284178,0.878110,00:42
4,0.088767,0.283394,0.881579,00:43
5,0.070382,0.285486,0.881818,00:42
6,0.062169,0.286824,0.882177,00:43


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

  self.pid = os.fork()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

  self.pid = os.fork()


In [44]:
learner.save("movie-classifier-stage-2")

Path('models/movie-classifier-stage-2.pth')

In [45]:
learner.export("models/movie-classifier-stage-2.pkl")

RuntimeError: File /content/drive/movie-classifier-stage-2.pkl cannot be opened.